In [8]:
from pathlib import Path

import pandas as pd

df = pd.read_csv('data/nih/Data_Entry_2017.csv')

In [3]:
disease_classes = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion'] # Effusion is Pleural Effusion

df['diseases'] = df['Finding Labels'].apply(lambda x: [d_class for d_class in disease_classes if d_class in x])
df['disease_count'] = df['Finding Labels'].apply(lambda x: sum(1 for d_class in disease_classes if d_class in x))

filtered_df = df[df['disease_count'] == 1]
filtered_df['disease'] = filtered_df['diseases'].apply(lambda x: x[0])

# Renaming Effusion to Pleural Effusion

filtered_df['disease'] = filtered_df['disease'].replace('Effusion', 'Pleural Effusion')
filtered_df['disease'].value_counts()

/var/folders/vj/mhdkxj391ljd_nk06v7m8hm40000gn/T/ipykernel_20883/1742232179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['disease'] = filtered_df['diseases'].apply(lambda x: x[0])
/var/folders/vj/mhdkxj391ljd_nk06v7m8hm40000gn/T/ipykernel_20883/1742232179.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['disease'] = filtered_df['disease'].replace('Effusion', 'Pleural Effusion')


disease
Pleural Effusion    8038
Atelectasis         7320
Consolidation       2516
Cardiomegaly        1436
Edema               1436
Name: count, dtype: int64

In [4]:
filtered_df


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,diseases,disease_count,disease
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143000,0.143000,NaN,[Cardiomegaly],1,Cardiomegaly
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143000,0.143000,NaN,[Cardiomegaly],1,Cardiomegaly
20,00000005_007.png,Effusion|Infiltration,7,5,70,F,PA,2566,2681,0.143000,0.143000,NaN,[Effusion],1,Pleural Effusion
23,00000008_000.png,Cardiomegaly,0,8,69,F,PA,2048,2500,0.171000,0.171000,NaN,[Cardiomegaly],1,Cardiomegaly
28,00000011_000.png,Effusion,0,11,75,M,PA,2638,2449,0.143000,0.143000,NaN,[Effusion],1,Pleural Effusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112084,00030780_000.png,Atelectasis,0,30780,67,M,PA,2021,2021,0.194311,0.194311,NaN,[Atelectasis],1,Atelectasis
112090,00030786_000.png,Effusion,0,30786,61,F,PA,2021,2021,0.194311,0.194311,NaN,[Effusion],1,Pleural Effusion
112091,00030786_001.png,Effusion|Emphysema|Pneumothorax,1,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,[Effusion],1,Pleural Effusion
112096,00030786_006.png,Consolidation,6,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,[Consolidation],1,Consolidation


In [5]:
# Randomly sample 1000 examples for each disease class
sampled_dfs = []
for disease in filtered_df['disease'].unique():
    disease_df = filtered_df[filtered_df['disease'] == disease]
    # Sample 1400
    n_samples = 1400
    sampled = disease_df.sample(n=n_samples, random_state=42)
    sampled_dfs.append(sampled)

# Combine all sampled dataframes
sampled_df = pd.concat(sampled_dfs, ignore_index=True)

# Shuffle the final dataframe
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Total sampled examples: {len(sampled_df)}")
print(f"\nClass distribution:")
print(sampled_df['disease'].value_counts())

Total sampled examples: 7000

Class distribution:
disease
Edema               1400
Atelectasis         1400
Pleural Effusion    1400
Cardiomegaly        1400
Consolidation       1400
Name: count, dtype: int64


In [9]:
nih_dir = Path('data/nih/')
image_file_dict = {file.name: file for file in nih_dir.glob('*/*/*.png')}

sampled_df['imgpath'] = sampled_df['Image Index'].map(image_file_dict)

# Add binary label columns for each disease
disease_columns = ['Edema', 'Atelectasis', 'Pleural Effusion', 'Cardiomegaly', 'Consolidation']
for disease in disease_columns:
    sampled_df[disease] = (sampled_df['disease'] == disease).astype(int)

sampled_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Unnamed: 11,diseases,disease_count,disease,imgpath,Edema,Atelectasis,Pleural Effusion,Cardiomegaly,Consolidation
0,00022470_009.png,Edema|Nodule,9,22470,46,M,AP,3056,2544,0.139,...,NaN,[Edema],1,Edema,data/nih/images_010/images/00022470_009.png,1,0,0,0,0
1,00004858_056.png,Atelectasis,56,4858,45,F,PA,2992,2991,0.143,...,NaN,[Atelectasis],1,Atelectasis,data/nih/images_003/images/00004858_056.png,0,1,0,0,0
2,00014626_023.png,Effusion|Infiltration,23,14626,44,F,AP,2692,2544,0.139,...,NaN,[Effusion],1,Pleural Effusion,data/nih/images_007/images/00014626_023.png,0,0,1,0,0
3,00016414_002.png,Cardiomegaly,2,16414,39,M,PA,2704,2781,0.143,...,NaN,[Cardiomegaly],1,Cardiomegaly,data/nih/images_008/images/00016414_002.png,0,0,0,1,0
4,00019805_005.png,Consolidation|Infiltration|Mass|Nodule,5,19805,29,M,AP,3056,2544,0.139,...,NaN,[Consolidation],1,Consolidation,data/nih/images_009/images/00019805_005.png,0,0,0,0,1


In [ ]:
sampled_df.to_csv('local_data/nih-sampled-meta.csv', index=False)